<a href="https://colab.research.google.com/github/kaisarmasum/Quantum-Machine-Learning/blob/main/Amplitude_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane
!pip install qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 45.3 MB/s eta 0:00:00
  Attempting uninstall: autograd
    Found existing installation: autograd 1.6.2
    Uninstalling autograd-1.6.2:
      Successfully uninstalled autograd-1.6.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pennylane as qml
from pennylane import numpy as np

def get_ab(x_in):
    x_new = np.reshape(x_in, (int(len(x_in) / 2), 2))
    i_max = int(len(x_new))
    j_max = int(np.ceil(np.log2(len(x_in))))

    P = np.ndarray((i_max, j_max), dtype=complex)
    alpha = np.ndarray((i_max, j_max), dtype=complex)
    beta = np.ndarray((i_max, j_max), dtype=complex)

    for j in range(j_max):
        for i, x in enumerate(x_new):
            if j == 0:
                p = np.power(np.linalg.norm(x), 2)
                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = x[0] / np.linalg.norm(x)
                    b = x[1] / np.linalg.norm(x)
            elif i >= i_max / (2 ** j):
                p = 0
                a = 1
                b = 0
            else:
                p = P[2 * i, j - 1] + P[2 * i + 1, j - 1]
                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = np.sqrt(P[2 * i, j - 1] / p)
                    b = np.sqrt(P[2 * i + 1, j - 1] / p)

            P[i, j] = p
            alpha[i, j] = a
            beta[i, j] = b

    return (alpha, beta)

def get_params(alpha, beta):
    alpha_mag = np.abs(alpha)
    alpha_phase = np.angle(alpha)
    beta_mag = np.abs(beta)
    beta_phase = np.angle(beta)

    with np.errstate(divide='ignore'):
        theta = 2 * np.arctan(beta_mag / alpha_mag)
    phi = beta_phase - alpha_phase
    r = np.sqrt(alpha_mag ** 2 + beta_mag ** 2)
    t = beta_phase + alpha_phase

    return theta, phi, r, t

def input_data(x_in):
    return get_params(*get_ab(x_in))

# Example usage
x_input = np.array([0.5, 0.5, 0.5, 0.5])  # Replace with your input data
theta, phi, r, t = input_data(x_input)

In [ ]:
# Define the number of qubits
num_qubits = 4 # Replace with your desired number of qubits

# Create a PennyLane quantum device
dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def circuit(theta, phi, t):
    for j in reversed(range(num_qubits)):
        n_j = num_qubits - 1 - j
        i_max = 2 ** (n_j)

        theta_j = theta[:i_max, j]

        if j != 0:
            for k in range(j + 1, num_qubits):
                qml.CRY(theta_j[k - (j + 1)], wires=[j, k])
        else:
            phi_j = phi[:i_max, j]
            t_j = t[:i_max, j]

            if phi_j.any() and t_j.any():
                for idx in range(len(theta_j)):
                    for k in range(j + 1, num_qubits):
                        qml.CRY(theta_j[idx], wires=[j, k])
                    Uij(theta_j[idx], phi_j[idx], t_j[idx], j)
            else:
                for k in range(j + 1, num_qubits):
                    qml.CRY(theta_j[k - (j + 1)], wires=[j, k])

    return qml.expval(qml.PauliZ(0))  # You can replace the Identity operator with your desired observable

def Ry(theta):
    return np.array([[np.cos(theta / 2), -np.sin(theta / 2)], [np.sin(theta / 2), np.cos(theta / 2)]])

def Rz(phi):
    return np.array([[np.exp(-1j * phi / 2), 0], [0, np.exp(1j * phi / 2)]])

# Define Uij gate
def Uij(theta, phi, t, wire):
    qml.RZ(-t, wires=wire)
    qml.RY(theta, wires=wire)
    qml.RZ(phi, wires=wire)

In [ ]:
# Define a function to calculate alpha and beta
def get_ab(x_in):
    x_new = np.reshape(x_in, (int(len(x_in) / 2), 2))
    i_max = int(len(x_new))
    j_max = int(np.ceil(np.log2(len(x_in))))

    P = np.ndarray((i_max, j_max), dtype=complex)
    alpha = np.ndarray((i_max, j_max), dtype=complex)
    beta = np.ndarray((i_max, j_max), dtype=complex)

    for j in range(j_max):
        for i, x in enumerate(x_new):
            if j == 0:
                p = np.linalg.norm(x) ** 2
                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = x[0] / np.linalg.norm(x)
                    b = x[1] / np.linalg.norm(x)
            elif i >= i_max / (2 ** j):
                p = 0
                a = 1
                b = 0
            else:
                p = P[2 * i, j - 1] + P[2 * i + 1, j - 1]
                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = np.sqrt(P[2 * i, j - 1] / p)
                    b = np.sqrt(P[2 * i + 1, j - 1] / p)

            P[i, j] = p
            alpha[i, j] = a
            beta[i, j] = b

    return (alpha, beta)

# Define a function to calculate theta, phi, r, and t
def get_params(alpha, beta):
    alpha_mag = np.abs(alpha)
    alpha_phase = np.angle(alpha)
    beta_mag = np.abs(beta)
    beta_phase = np.angle(beta)

    with np.errstate(divide='ignore'):
        theta = 2 * np.arctan(beta_mag / alpha_mag)
    phi = beta_phase - alpha_phase
    r = np.sqrt(alpha_mag ** 2 + beta_mag ** 2)
    t = beta_phase + alpha_phase

    return theta, phi, r, t

# Define the number of qubits and parameter arrays theta, phi, and t
x_input = ([1.9, 0.9, 0.3, 0.4, 0.5, 0.0, 0.7, 0.0, 0.9, 1.0, 1.1, 1.2])  # Replace with your input data
alpha, beta = get_ab(x_input)
theta, phi, r, t = get_params(alpha, beta)

result = circuit(theta, phi, t)
print("Expectation value:", result)

Expectation value: 1.0


In [ ]:
import pennylane as qml
from pennylane import numpy as np

# Define the number of qubits
num_qubits = 4  # Replace with your desired number of qubits

# Create a PennyLane quantum device
dev = qml.device("default.qubit", wires=num_qubits)

# Create an empty list to store the state vectors
state_vectors = []

@qml.qnode(dev)
def circuit(theta, phi, t):
    for j in reversed(range(num_qubits)):
        n_j = num_qubits - 1 - j
        i_max = 2 ** (n_j)

        theta_j = theta[:i_max, j]

        if j != 0:
            for k in range(j + 1, num_qubits):
                qml.CRY(theta_j[k - (j + 1)], wires=[j, k])
        else:
            phi_j = phi[:i_max, j]
            t_j = t[:i_max, j]

            if phi_j.any() and t_j.any():
                for idx in range(len(theta_j)):
                    for k in range(j + 1, num_qubits):
                        qml.CRY(theta_j[idx], wires=[j, k])
                    Uij(theta_j[idx], phi_j[idx], t_j[idx], j)
            else:
                for k in range(j + 1, num_qubits):
                    qml.CRY(theta_j[k - (j + 1)], wires=[j, k])

    # Store the state vector in the list
    state_vectors.append(dev._state)

    # Return a measurement to satisfy the QNode requirements
    return qml.expval(qml.PauliZ(0))  # You can replace this with any measurement

def Ry(theta):
    return np.array([[np.cos(theta / 2), -np.sin(theta / 2)], [np.sin(theta / 2), np.cos(theta / 2)]])

def Rz(phi):
    return np.array([[np.exp(-1j * phi / 2), 0], [0, np.exp(1j * phi / 2)]])

# Define Uij gate
def Uij(theta, phi, t, wire):
    qml.RZ(-t, wires=wire)
    qml.RY(theta, wires=wire)
    qml.RZ(phi, wires=wire)

# Define a function to calculate alpha and beta
def get_ab(x_in):
    x_new = np.reshape(x_in, (int(len(x_in) / 2), 2))
    i_max = int(len(x_new))
    j_max = int(np.ceil(np.log2(len(x_in))))

    P = np.ndarray((i_max, j_max), dtype=complex)
    alpha = np.ndarray((i_max, j_max), dtype=complex)
    beta = np.ndarray((i_max, j_max), dtype=complex)

    for j in range(j_max):
        for i, x in enumerate(x_new):
            if j == 0:
                p = np.linalg.norm(x) ** 2
                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = x[0] / np.linalg.norm(x)
                    b = x[1] / np.linalg.norm(x)
            elif i >= i_max / (2 ** j):
                p = 0
                a = 1
                b = 0
            else:
                p = P[2 * i, j - 1] + P[2 * i + 1, j - 1]
                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = np.sqrt(P[2 * i, j - 1] / p)
                    b = np.sqrt(P[2 * i + 1, j - 1] / p)

            P[i, j] = p
            alpha[i, j] = a
            beta[i, j] = b

    return (alpha, beta)

# Define a function to calculate theta, phi, r, and t
def get_params(alpha, beta):
    alpha_mag = np.abs(alpha)
    alpha_phase = np.angle(alpha)
    beta_mag = np.abs(beta)
    beta_phase = np.angle(beta)

    with np.errstate(divide='ignore'):
        theta = 2 * np.arctan(beta_mag / alpha_mag)
    phi = beta_phase - alpha_phase
    r = np.sqrt(alpha_mag ** 2 * beta_mag ** 2)
    t = beta_phase + alpha_phase

    return theta, phi, r, t

# Define the number of qubits and parameter arrays theta, phi, and t
x_input = ([1.9, 0.9, 0.3, 0.4, 0.5, 0.0, 0.7, 0.0, 0.9, 1.0, 1.1, 1.2])  # Replace with your input data
alpha, beta = get_ab(x_input)
theta, phi, r, t = get_params(alpha, beta)

# Execute the circuit to generate the state vector
circuit(theta, phi, t)

# Access and print the state vector of the entire circuit
state_vector_circuit = dev._state
print("State vector of the entire circuit:\n", state_vector_circuit)


State vector of the entire circuit:
 [[[[1.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]]


 [[[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]]]


In [ ]:
# Define the number of qubits and parameter arrays theta, phi, and t
x_input = np.array([[1.9, 0.9, 0.3, 0.4, 0.5, 0.0, 0.7, 0.0, 0.9, 1.0, 1.1, 1.2],
                   [0.5, 0.7, 0.2, 0.3, 0.8, 0.1, 0.6, 0.4, 0.9, 0.5, 0.3, 0.6]])

# Create a list to store state vectors
state_vectors = []

for params in x_input:
    alpha, beta = get_ab(params)
    theta, phi, r, t = get_params(alpha, beta)

    # Execute the circuit to generate the state vector
    circuit(theta, phi, t)

    # Access and store the state vector of the entire circuit
    state_vector_circuit = dev._state
    state_vectors.append(state_vector_circuit)

# Print the state vectors for each set of parameters
for i, state_vector in enumerate(state_vectors):
    print(f"State vector {i + 1}:\n", state_vector)

State vector 1:
 [[[[1.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]]


 [[[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]]]
State vector 2:
 [[[[1.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]]


 [[[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]]]
State vector 3:
 [[[[1.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]]


 [[[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]]]
State vector 4:
 [[[[1.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]]


 [[[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]]]


In [ ]:
import pennylane as qml
from pennylane import numpy as np

# Define the number of qubits
num_qubits = 4  # Replace with your desired number of qubits

# Create a PennyLane quantum device
dev = qml.device("default.qubit", wires=num_qubits)

# Create an empty list to store the state vectors
state_vectors = []

@qml.qnode(dev)
def circuit(theta, phi, t):
    for j in reversed(range(num_qubits)):
        n_j = num_qubits - 1 - j
        i_max = 2 ** (n_j)

        theta_j = theta[:i_max, j]

        if j != 0:
            for k in range(j + 1, num_qubits):
                qml.CRY(theta_j[k - (j + 1)], wires=[j, k])
        else:
            phi_j = phi[:i_max, j]
            t_j = t[:i_max, j]

            if phi_j.any() and t_j.any():
                for idx in range(len(theta_j)):
                    for k in range(j + 1, num_qubits):
                        qml.CRY(theta_j[idx], wires=[j, k])
                    Uij(theta_j[idx], phi_j[idx], t_j[idx], j)
            else:
                for k in range(j + 1, num_qubits):
                    qml.CRY(theta_j[k - (j + 1)], wires=[j, k])

    # Store the state vector in the list
    state_vectors.append(dev._state)

    # Return a measurement to satisfy the QNode requirements
    return qml.expval(qml.PauliZ(0))  # You can replace this with any measurement

def Ry(theta):
    return np.array([[np.cos(theta / 2), -np.sin(theta / 2)], [np.sin(theta / 2), np.cos(theta / 2)]])

def Rz(phi):
    return np.array([[np.exp(-1j * phi / 2), 0], [0, np.exp(1j * phi / 2)]])

# Define Uij gate
def Uij(theta, phi, t, wire):
    qml.RZ(-t, wires=wire)
    qml.RY(theta, wires=wire)
    qml.RZ(phi, wires=wire)

# Define a function to calculate alpha and beta
def get_ab(x_in):
    x_new = np.reshape(x_in, (int(len(x_in) / 2), 2))
    i_max = int(len(x_new))
    j_max = int(np.ceil(np.log2(len(x_in))))

    P = np.ndarray((i_max, j_max), dtype=complex)
    alpha = np.ndarray((i_max, j_max), dtype=complex)
    beta = np.ndarray((i_max, j_max), dtype=complex)

    for j in range(j_max):
        for i, x in enumerate(x_new):
            if j == 0:
                p = np.linalg.norm(x) ** 2
                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = x[0] / np.linalg.norm(x)
                    b = x[1] / np.linalg.norm(x)
            elif i >= i_max / (2 ** j):
                p = 0
                a = 1
                b = 0
            else:
                p = P[2 * i, j - 1] + P[2 * i + 1, j - 1]
                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = np.sqrt(P[2 * i, j - 1] / p)
                    b = np.sqrt(P[2 * i + 1, j - 1] / p)

            P[i, j] = p
            alpha[i, j] = a
            beta[i, j] = b

    return (alpha, beta)

# Define a function to calculate theta, phi, r, and t
def get_params(alpha, beta):
    alpha_mag = np.abs(alpha)
    alpha_phase = np.angle(alpha)
    beta_mag = np.abs(beta)
    beta_phase = np.angle(beta)

    with np.errstate(divide='ignore'):
        theta = 2 * np.arctan(beta_mag / alpha_mag)
    phi = beta_phase - alpha_phase
    r = np.sqrt(alpha_mag ** 2 * beta_mag ** 2)
    t = beta_phase + alpha_phase

    return theta, phi, r, t

# Define the number of qubits and parameter arrays theta, phi, and t
x_input = ([1.9, 0.9, 0.3, 0.4, 0.5, 0.0, 0.7, 0.0, 0.9, 1.0, 1.1, 1.2])  # Replace with your input data
alpha, beta = get_ab(x_input)
theta, phi, r, t = get_params(alpha, beta)

# Execute the circuit to generate the state vector
circuit(theta, phi, t)

# Access and print the state vector of the entire circuit
state_vector_circuit = dev._state
print("State vector of the entire circuit:")

# Print the state vector for a 4-qubit circuit
for i, amplitude in enumerate(state_vector_circuit):
    print(f"Amplitude for state {bin(i)[2:].zfill(4)}: {amplitude}")

State vector of the entire circuit:
Amplitude for state 0000: [[[1.+0.j 0.+0.j]
  [0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j]]]
Amplitude for state 0001: [[[0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j]]]


In [ ]:
import pennylane as qml
from pennylane import numpy as np

# Define the number of qubits
num_qubits = 4

# Create a PennyLane quantum device
dev = qml.device("default.qubit", wires=num_qubits)

# Create an empty list to store the state vectors
state_vectors = []

@qml.qnode(dev)
def circuit(theta, phi, t):
    for j in reversed(range(num_qubits)):
        n_j = num_qubits - 1 - j
        i_max = 2 ** (n_j)

        theta_j = theta[:i_max, j]

        if j != 0:
            for k in range(j + 1, num_qubits):
                qml.CRY(theta_j[k - (j + 1)], wires=[j, k])
        else:
            phi_j = phi[:i_max, j]
            t_j = t[:i_max, j]

            if phi_j.any() and t_j.any():
                for idx in range(len(theta_j)):
                    for k in range(j + 1, num_qubits):
                        qml.CRY(theta_j[idx], wires=[j, k])
                    Uij(theta_j[idx], phi_j[idx], t_j[idx], j)
            else:
                for k in range(j + 1, num_qubits):
                    qml.CRY(theta_j[k - (j + 1)], wires=[j, k])

    # Store the state vector in the list
    state_vectors.append(dev._state)

    # Return a measurement to satisfy the QNode requirements
    return qml.expval(qml.PauliZ(0))  # You can replace this with any measurement

def Ry(theta):
    return np.array([[np.cos(theta / 2), -np.sin(theta / 2)], [np.sin(theta / 2), np.cos(theta / 2)]])

def Rz(phi):
    return np.array([[np.exp(-1j * phi / 2), 0], [0, np.exp(1j * phi / 2)]])

# Define Uij gate
def Uij(theta, phi, t, wire):
    qml.RZ(-t, wires=wire)
    qml.RY(theta, wires=wire)
    qml.RZ(phi, wires=wire)

# Define a function to calculate alpha and beta
def get_ab(x_in):
    x_new = np.reshape(x_in, (int(len(x_in) / 2), 2))
    i_max = int(len(x_new))
    j_max = int(np.ceil(np.log2(len(x_in))))

    P = np.ndarray((i_max, j_max), dtype=complex)
    alpha = np.ndarray((i_max, j_max), dtype=complex)
    beta = np.ndarray((i_max, j_max), dtype=complex)

    for j in range(j_max):
        for i, x in enumerate(x_new):
            if j == 0:
                p = np.linalg.norm(x) ** 2
                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = x[0] / np.linalg.norm(x)
                    b = x[1] / np.linalg.norm(x)
            elif i >= i_max / (2 ** j):
                p = 0
                a = 1
                b = 0
            else:
                p = P[2 * i, j - 1] + P[2 * i + 1, j - 1]
                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = np.sqrt(P[2 * i, j - 1] / p)
                    b = np.sqrt(P[2 * i + 1, j - 1] / p)

            P[i, j] = p
            alpha[i, j] = a
            beta[i, j] = b

    return (alpha, beta)

# Define a function to calculate theta, phi, r, and t
def get_params(alpha, beta):
    alpha_mag = np.abs(alpha)
    alpha_phase = np.angle(alpha)
    beta_mag = np.abs(beta)
    beta_phase = np.angle(beta)

    with np.errstate(divide='ignore'):
        theta = 2 * np.arctan(beta_mag / alpha_mag)
    phi = beta_phase - alpha_phase
    r = np.sqrt(alpha_mag ** 2 * beta_mag ** 2)
    t = beta_phase + alpha_phase

    return theta, phi, r, t

# Define the number of qubits and parameter arrays theta, phi, and t
x_input = ([1.9, 0.9, 0.3, 0.4, 0.5, 0.0, 0.7, 0.0, 0.9, 1.0, 1.1, 1.2])  # Replace with your input data
alpha, beta = get_ab(x_input)
theta, phi, r, t = get_params(alpha, beta)

# Execute the circuit to generate the state vector
circuit(theta, phi, t)

# Access and print the state vector of the entire circuit
state_vector_circuit = state_vectors[-1]  # Get the final state vector
print("State vector of the entire circuit:")

# Print the state vector for a 4-qubit circuit
for i, amplitude in enumerate(state_vector_circuit):
    print(f"Amplitude for state {bin(i)[2:].zfill(4)}: {amplitude}")

State vector of the entire circuit:
Amplitude for state 0000: [[[1.+0.j 0.+0.j]
  [0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j]]]
Amplitude for state 0001: [[[0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j]]]


In [ ]:
import pennylane as qml
from pennylane import numpy as np

# Define the number of qubits
num_qubits = 4

# Create a PennyLane quantum device
dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def circuit(theta, phi, t):
    # Initialize the state to |0> for all qubits
    for wire in range(num_qubits):
        qml.RY(0.0, wires=wire)

    for j in reversed(range(num_qubits)):
        n_j = num_qubits - 1 - j
        i_max = 2 ** (n_j)

        theta_j = theta[:i_max, j]

        if j != 0:
            for k in range(j + 1, num_qubits):
                qml.CRY(theta_j[k - (j + 1)], wires=[j, k])
        else:
            phi_j = phi[:i_max, j]
            t_j = t[:i_max, j]

            if phi_j.any() and t_j.any():
                for idx in range(len(theta_j)):
                    for k in range(j + 1, num_qubits):
                        qml.CRY(theta_j[idx], wires=[j, k])
                    Uij(theta_j[idx], phi_j[idx], t_j[idx], j)
            else:
                for k in range(j + 1, num_qubits):
                    qml.CRY(theta_j[k - (j + 1)], wires=[j, k])

    # Return the state vector
    return qml.state()

def Ry(theta):
    return np.array([[np.cos(theta / 2), -np.sin(theta / 2)], [np.sin(theta / 2), np.cos(theta / 2)]])

def Rz(phi):
    return np.array([[np.exp(-1j * phi / 2), 0], [0, np.exp(1j * phi / 2)]])

# Define Uij gate
def Uij(theta, phi, t, wire):
    qml.RZ(-t, wires=wire)
    qml.RY(theta, wires=wire)
    qml.RZ(phi, wires=wire)

# Define a function to calculate alpha and beta
def get_ab(x_in):
    x_new = np.reshape(x_in, (int(len(x_in) / 2), 2))
    i_max = int(len(x_new))
    j_max = int(np.ceil(np.log2(len(x_in))))

    P = np.ndarray((i_max, j_max), dtype=complex)
    alpha = np.ndarray((i_max, j_max), dtype=complex)
    beta = np.ndarray((i_max, j_max), dtype=complex)

    for j in range(j_max):
        for i, x in enumerate(x_new):
            if j == 0:
                p = np.linalg.norm(x) ** 2
                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = x[0] / np.linalg.norm(x)
                    b = x[1] / np.linalg.norm(x)
            elif i >= i_max / (2 ** j):
                p = 0
                a = 1
                b = 0
            else:
                p = P[2 * i, j - 1] + P[2 * i + 1, j - 1]
                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = np.sqrt(P[2 * i, j - 1] / p)
                    b = np.sqrt(P[2 * i + 1, j - 1] / p)

            P[i, j] = p
            alpha[i, j] = a
            beta[i, j] = b

    return (alpha, beta)

# Define a function to calculate theta, phi, r, and t
def get_params(alpha, beta):
    alpha_mag = np.abs(alpha)
    alpha_phase = np.angle(alpha)
    beta_mag = np.abs(beta)
    beta_phase = np.angle(beta)

    with np.errstate(divide='ignore'):
        theta = 2 * np.arctan(beta_mag / alpha_mag)
    phi = beta_phase - alpha_phase
    r = np.sqrt(alpha_mag ** 2 * beta_mag ** 2)
    t = beta_phase + alpha_phase

    return theta, phi, r, t

# Define the number of qubits and parameter arrays theta, phi, and t
x_input = ([1.9, 0.9, 0.3, 0.4, 0.5, 0.0, 0.7, 0.0, 0.9, 1.0, 1.1, 1.2])  # Replace with your input data
alpha, beta = get_ab(x_input)
theta, phi, r, t = get_params(alpha, beta)

# Execute the circuit to generate the state vector
state_vector_circuit = circuit(theta, phi, t)

# Create all possible binary strings of 4 qubits
binary_states = [format(i, f'0{num_qubits}b') for i in range(2**num_qubits)]

# Print the state vector for all 16 states
for i, binary_str in enumerate(binary_states):
    amplitude = state_vector_circuit[i]
    print(f"State {binary_str}: Amplitude = {amplitude}")

State 0000: Amplitude = (1+0j)
State 0001: Amplitude = 0j
State 0010: Amplitude = 0j
State 0011: Amplitude = 0j
State 0100: Amplitude = 0j
State 0101: Amplitude = 0j
State 0110: Amplitude = 0j
State 0111: Amplitude = 0j
State 1000: Amplitude = 0j
State 1001: Amplitude = 0j
State 1010: Amplitude = 0j
State 1011: Amplitude = 0j
State 1100: Amplitude = 0j
State 1101: Amplitude = 0j
State 1110: Amplitude = 0j
State 1111: Amplitude = 0j


In [ ]:
# Define the number of qubits and parameter arrays theta, phi, and t
x_input = [0.2, 0.3, 0.1, 0.4, 0.7, 0.8, 0.9, 0.5, 0.4, 0.3, 0.2, 0.1]  # Replace with your new input data
alpha, beta = get_ab(x_input)
theta, phi, r, t = get_params(alpha, beta)

# Execute the circuit to generate the state vector
state_vector_circuit = circuit(theta, phi, t)

# Create all possible binary strings of 4 qubits
binary_states = [format(i, f'0{num_qubits}b') for i in range(2**num_qubits)]

# Print the state vector for all 16 states
for i, binary_str in enumerate(binary_states):
    amplitude = state_vector_circuit[i]
    print(f"State {binary_str}: Amplitude = {amplitude}")

State 0000: Amplitude = (1+0j)
State 0001: Amplitude = 0j
State 0010: Amplitude = 0j
State 0011: Amplitude = 0j
State 0100: Amplitude = 0j
State 0101: Amplitude = 0j
State 0110: Amplitude = 0j
State 0111: Amplitude = 0j
State 1000: Amplitude = 0j
State 1001: Amplitude = 0j
State 1010: Amplitude = 0j
State 1011: Amplitude = 0j
State 1100: Amplitude = 0j
State 1101: Amplitude = 0j
State 1110: Amplitude = 0j
State 1111: Amplitude = 0j


In [ ]:
|